In [2]:
%pip install pandas
%pip install numpy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

data = pd.read_excel('hasil/klasifikasi_sentimen_perasaan.xlsx')
data

C:\Users\halim\AppData\Local\Temp\ipykernel_14804\3296562847.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,text,rating,polarity_emoticon,polarity_text,text_join,polarity,sentimen
0,Aplikasinya bagus semua nya bagus tapi tolong ...,5,1,-16,aplikasi bagus nya bagus tolong tambah teman a...,-15,marah
1,Lewat aplikasi ruang guru saya bisa tau pembel...,5,0,-6,aplikasi ruang guru ajar pembelajaranya erti,-6,sangat marah
2,Seru isinya bisa ngerjain tugas dr aplikasi at...,5,0,-5,seru isi tugas aplikasi guru ngajar,-5,sangat sedih
3,Jgn di dwnld Nanti bayar,3,0,-2,dwnld bayar,-2,sedih
4,Apk dan webnya sering error tapi pengajarannya...,3,0,-8,apk webnya error ajar bantu paham materi,-8,marah
...,...,...,...,...,...,...,...
13443,Bagus banget,5,0,-2,bagus,-2,sedih
13444,Keren,5,0,0,keren,0,sedih
13445,gw ngga ngerti cara mainnya,2,0,0,erti main,0,sedih
13446,Paket 3 bulan Explorers hanya masuk 3x mengapa...,1,0,6,paket explorers masuk luar duga,6,bahagia


# Naive Bayes - TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def sentiment_classification_nb(dataset, target, test_size=0.2, method='multinomial', alpha=1.0):
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=test_size, random_state=42)
    
    # Create and fit a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    
    # Naive Bayes model selection
    if method == 'gaussian':
        # GaussianNB requires dense input
        X_train_dense = X_train_tfidf.toarray()
        model = GaussianNB()
    elif method == 'multinomial':
        model = MultinomialNB(alpha=alpha)
    elif method == 'bernoulli':
        model = BernoulliNB(alpha=alpha)
    else:
        raise ValueError("Invalid method! Choose from 'gaussian', 'multinomial', or 'bernoulli'.")
    
    # Fit the model
    if method == 'gaussian':
        model.fit(X_train_dense, y_train)
    else:
        model.fit(X_train_tfidf, y_train)
    
    # Transform the test data and predict
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    if method == 'gaussian':
        y_pred = model.predict(X_test_tfidf.toarray())
    else:
        y_pred = model.predict(X_test_tfidf)
    
    # Calculate and return accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Example usage assuming 'data' is a DataFrame with 'text_join' and 'sentimen' columns
dataset = data['text_join']
target = data['sentimen']

# Multinomial Naive Bayes
accuracy_multinomial = sentiment_classification_nb(dataset, target, method='multinomial', alpha=1.0)
print("Multinomial NB Accuracy:", accuracy_multinomial)

# Gaussian Naive Bayes
accuracy_gaussian = sentiment_classification_nb(dataset, target, method='gaussian')
print("Gaussian NB Accuracy:", accuracy_gaussian)

# Bernoulli Naive Bayes
accuracy_bernoulli = sentiment_classification_nb(dataset, target, method='bernoulli', alpha=1.0)
print("Bernoulli NB Accuracy:", accuracy_bernoulli)


Multinomial NB Accuracy: 0.528996282527881
Gaussian NB Accuracy: 0.08661710037174722
Bernoulli NB Accuracy: 0.5074349442379182


# Support Vector Machine - TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def sentiment_classification_svm(dataset, target, test_size=0.2, kernel='linear', degree=3):
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(dataset, target, test_size=test_size, random_state=42)
    
    # Create TF-IDF vectorizer and transform the training data
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    
    # Initialize the SVM model with the specified kernel and degree
    model = svm.SVC(kernel=kernel, degree=degree)
    
    # Train the SVM model
    model.fit(X_train_tfidf, y_train)
    
    # Transform the testing data and predict using the trained model
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    y_pred = model.predict(X_test_tfidf)
    
    # Calculate and return the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Example usage:
dataset = data['text_join']  # Assume 'data' is a DataFrame with text data
target = data['sentimen']  # Assume 'data' has a sentiment column

# Linear SVM
accuracy_linear = sentiment_classification_svm(dataset, target, kernel='linear')
print("Linear SVM Accuracy:", accuracy_linear)

# Polynomial SVM
accuracy_poly = sentiment_classification_svm(dataset, target, kernel='poly')
print("Polynomial SVM Accuracy:", accuracy_poly)

# RBF SVM
accuracy_rbf = sentiment_classification_svm(dataset, target, kernel='rbf')
print("RBF SVM Accuracy:", accuracy_rbf)


Linear SVM Accuracy: 0.6717472118959108
Polynomial SVM Accuracy: 0.6104089219330855
RBF SVM Accuracy: 0.6698884758364312
